In [1]:
from init import *
%matplotlib qt

In [2]:
N = 2000
np.random.seed(1)
m = np.random.lognormal(np.log(3.72), 0.44, N)
np.random.seed(2)
xo = np.random.lognormal(np.log(0.02), 0.71, N)
np.random.seed(3)
p = broken_power_per_sampler(5.75, 2.31, 0.08, N)
a = per2sma(p)
T = 279/a**0.5
rc = m**(1/3.7)
t = np.geomspace(100, 5000, 100)

In [3]:
print(pd.DataFrame({'m': m, 'xo': xo, 'p': p, 'rc': rc}))

             m        xo          p        rc
0     7.602184  0.014877  30.103347  1.730181
1     2.842122  0.019217  46.596863  1.326191
2     2.948593  0.004389  13.575984  1.339439
3     2.320117  0.064092  26.837020  1.255412
4     5.443940  0.005598  76.025417  1.580868
...        ...       ...        ...       ...
1995  4.041840  0.023448  14.412907  1.458614
1996  4.761333  0.020151   2.984453  1.524649
1997  2.479839  0.037054  10.102993  1.278206
1998  4.945948  0.018342  20.964920  1.540405
1999  6.847797  0.034215  59.554934  1.681994

[2000 rows x 4 columns]


# Run photoevaporation with LF14 (Lopez & Fortney 2014)

In [9]:
rpo = rad_lopezfortney_analytic(m, rc, xo, T, age=t[0]) # Get the radius before photoevaporation for plotting
rp, x = atm_structure_photevap(m, xo, T, a, rc=rc, eta=0.1, t=t, rp_calc_meth='LF14')  # Really fast!
bins = fulton_redges(0.9, 6)
plt.figure(figsize=(10, 7))
plt.hist(rpo, bins, histtype='step', label='Before photoevaporation')
plt.hist(rp, bins, histtype='step', label='After photoevaporation')
plt.ylabel('Count', size=20)
plt.xlabel(f'Radius ({runit})', size=20)
plt.legend(fontsize=16)

# Run photoevaporation with evapmass (Owen & Estrada 2020, Owen & Wu 2017)

In [5]:
# Import evapmass
import sys
# Add path of the evapmass package
evapmass_path = '../../EvapMass-master'
sys.path.append(evapmass_path)
from planet_structure import solve_structure

- "total_rad_generic()" allows custom function
- Arguments needed by the custom function has to be defined:
    * 1. The variable arguments should be one of: 'mp' (for mass), 'X' (for atmospheric mass-fraction), 'Teq' (for equilibrium temperature), 'rc' (for core radius in RE), 'wf' (for WMF), and 'age' (for age!)
    * 2. Any fixed argument can be defined arbitrarily and that has to be parsed to atm_structure_photevap() as a keyworded argument, e.g. atm_structure_photevap(..., Xiron=0)


In [6]:
# Define custom function: 
def calc_radii_using_evapmass(mp, X, Teq, age=None, wf=0, Tkh_Myr=100, Xiron=0.3):
    mp = np.atleast_1d(mp)
    mp, X, Teq, Xiron, age, wf = np.broadcast_arrays(mp, X, Teq, Xiron, age, wf)
    # Tkh_Myr = Tkh_Myr if age < 100 else age
    rp = np.zeros_like(mp)
    for i in range(len(mp)):
        rp[i] = solve_structure(X[i], Teq[i], mp[i], Tkh_Myr, Xiron[i], wf[i])[2][0] / RE  # The second output is total radius and it is in cgs unit
    return rp

# Define arguments needed
args_for_evapmass = 'mp', 'X', 'Teq', 'age', 'wf', 'Tkh_Myr', 'Xiron'

In [7]:
# A bit slow :(
rp1, x1 = atm_structure_photevap(m, xo, T, a, rc=rc, t=t, rp_calc_meth=calc_radii_using_evapmass, args=args_for_evapmass, progress=True, wf=0, Tkh_Myr=100, Xiron=0.3)

time-step: 99/99


In [21]:
fig, ax = plt.subplots(1, 2, figsize=(15, 7))
ax[0].hist(rp, bins, histtype='step', label='After photoevaporation (LF+14)')
ax[0].hist(rp1, bins, histtype='step', label='After photoevaporation (Evapmass)')
ax[0].set_ylabel('Count', size=20)
ax[0].set_xlabel(f'Radius ({runit})', size=20)

Xbins = np.geomspace(np.min([xo, x, x1]), np.max([xo, x, x1]), 50)
ax[1].hist(x, Xbins, histtype='step', label='After photoevaporation (LF+14)')
ax[1].hist(x1, Xbins, histtype='step', label='After photoevaporation (Evapmass)')
ax[1].hist(xo, Xbins, histtype='step', label='Before photoevaporation')
ax[1].set_xlabel(f'Atmospheric mass-fraction', size=20)
ax[1].set_xscale('log')
ax[1].legend(fontsize=16)
fig.tight_layout()

In [12]:
np.min([xo, x, x1]), np.max([xo, x, x1])

(1e-05, 0.3697784477605486)